### I used pandas, numpy, requests, beautifulsoup to web scrape the wikipage 

In [88]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from tabulate import tabulate

In [89]:
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(page, 'html.parser')
table = soup.find('table', class_='sortable')
row = []
for tr in table.find_all('tr'):                                           
    if tr.find_all('th') == []:                                           
        row.append([td.get_text(strip=True) for td in tr.find_all('td')]) 
column_names = ['Postcode', 'Borough', 'Neighborhood']

df = pd.DataFrame(row, columns=column_names) 

#drop NaN Borough
drop_that = df[df['Borough'] == 'Not assigned'].index
df2 = df.drop(drop_that, axis=0)                          
df2.reset_index(drop=True, inplace=True)

#Merged
nh_na = df2[df2['Neighborhood'] == 'Not assigned'].index 
df2.iloc[nh_na, 2] = df2['Borough'][nh_na]


df2 = df2.groupby(['Postcode', 'Borough'], as_index=False).agg(lambda x: ', '.join(x))
df2


,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [90]:
df2.shape

(103, 3)

# Adding the latitude and longitude on the grid

In [91]:
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!pip install folium
import folium
!pip install geocoder
import geocoder

In [92]:
lat = []
lng = []
lat_lng_coords = None

postal_code = neighborhoods['Postcode']

for pc in postal_code:
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(pc))
    lat_lng_coords = g.latlng
    lat.append(lat_lng_coords[0])
    lng.append(lat_lng_coords[1])
   
df3 = df2
df3['Latitude'] = lat
df3['Longitude'] = lng

In [93]:
df3

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785665,-79.158725
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765815,-79.175193
3,M1G,Scarborough,Woburn,43.768369,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440
5,M1J,Scarborough,Scarborough Village,43.743125,-79.231750
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.726276,-79.263625
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.713054,-79.285055
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.724235,-79.227925
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.696770,-79.259967


In [108]:
df4 = df3[df3['Borough'].str.find('Toronto') != -1].reset_index(drop=True)
print(df4.shape)
df4

(38, 5)


,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676531,-79.295425
1,M4K,East Toronto,"The Danforth West, Riverdale",43.683178,-79.355105
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.667965,-79.314667
3,M4M,East Toronto,Studio District,43.660629,-79.334855
4,M4N,Central Toronto,Lawrence Park,43.728420,-79.387133
5,M4P,Central Toronto,Davisville North,43.712755,-79.388514
6,M4R,Central Toronto,North Toronto West,43.714523,-79.406960
7,M4S,Central Toronto,Davisville,43.703395,-79.385964
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.690685,-79.382946
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686074,-79.402265


In [95]:
g = geocoder.arcgis('Toronto, Ontario')
lat_tor = g.latlng[0]
lng_tor = g.latlng[1]
Torontomap = folium.Map(location=[lat_tor, lng_tor], zoom_start=11)
for lat, lng, bor, postcode in zip(df4['Latitude'], df4['Longitude'], df4['Borough'], df4['Postcode']):
    label = '{}, {}'.format(postcode, bor)        # popup labels with postcode and borough
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng],
                        radius=5,
                        popup=label,
                        color='blue',
                        fill=True,
                        fill_color='#3186cc',
                        fill_opacity=0.7,
                        parse_html=False).add_to(Torontomap)
Torontomap

In [96]:
client_id='FQROVQZIVWZEWWMHFEVMNFBMOTPUDRBD2RTTLKZAYVGW4PC4'
client_secrect='LS2UHM2OU2BX3JDY4ZR04RMX4QJ0EH2MEUSLDQ0IXH4YSRMZ'
version='20170511'

# API Request for Venues

In [97]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):        
      
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(client_id, client_secrect,version,lat,lng,radius,LIMIT)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(name,lat,lng, v['venue']['name'],v['venue']['location']['lat'],v['venue']['location']['lng'],v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode','Latitude','Longitude','Venue','Venue Latitude','Venue Longitude','Venue Category']
    
    return(nearby_venues)

LIMIT = 100
TorontoVenues = getNearbyVenues(names=df4['Postcode'], latitudes=df4['Latitude'],longitudes=df4['Longitude'],radius=700) 

In [99]:
print(TorontoVenues.shape)
TorontoVenues.head()

(2369, 7)


,Postcode,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4E,43.676531,-79.295425,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,43.676531,-79.295425,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,43.676531,-79.295425,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
3,M4E,43.676531,-79.295425,The Beech Tree,43.680493,-79.288846,Gastropub
4,M4E,43.676531,-79.295425,Beaches Bake Shop,43.680363,-79.289692,Bakery


# Total Venues for Each Postcode

In [100]:
totalvenues = pd.DataFrame()
totalvenues['Postcode'] = TorontoVenues.groupby('Postcode').count().reset_index()['Postcode']
totalvenues['Venue Count'] = TorontoVenues.groupby('Postcode').count().reset_index()['Venue']
totalvenues

,Postcode,Venue Count
0,M4E,29
1,M4K,29
2,M4L,34
3,M4M,82
4,M4N,7
5,M4P,26
6,M4R,37
7,M4S,43
8,M4T,8
9,M4V,24


# Onehot Dummy Values for Cluster

In [101]:
toronto_onehot = pd.get_dummies(TorontoVenues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Postcode'] = TorontoVenues['Postcode'] 

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Postcode').mean().reset_index()
toronto_grouped.head()

,Postcode,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Transportation Service,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,M4E,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.034483,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
1,M4K,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.034483
2,M4L,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
3,M4M,0.0,0.0,0.0,0.036585,0.012195,0.0,0.02439,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.012195,0.0,0.012195,0.0,0.012195
4,M4N,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000


# Most Common Venues 

In [102]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]                                       
    row_categories_sorted = row_categories.sort_values(ascending=False) 
    
    return row_categories_sorted.index.values[0:num_top_venues]
num_top_venues = 10

indicators = ['st', 'nd', 'rd']


columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind])) # 1st, 2nd, 3rd
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))


toronto_venues_sorted = pd.DataFrame(columns=columns)
toronto_venues_sorted['Postcode'] = toronto_grouped['Postcode']


for ind in np.arange(toronto_grouped.shape[0]):
    toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

toronto_venues_sorted.head()

# Settting the Numbers of Clusters

In [104]:
kclusters = 3
toronto_grouped_clustering = toronto_grouped.drop('Postcode', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0], dtype=int32)

# Merging the Tables

In [105]:
toronto_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

df5 = df4

# Merge toronto_grouped with toronto to add latitude/longitude for each neighborhood
df5 = df5.join(toronto_venues_sorted.set_index('Postcode'), on='Postcode')
df5[['Cluster Labels']] = df5[['Cluster Labels']].astype('int')

df5.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676531,-79.295425,0,Japanese Restaurant,Pizza Place,Bar,Health Food Store,Coffee Shop,Gastropub,Neighborhood,Juice Bar,Pub,Breakfast Spot
1,M4K,East Toronto,"The Danforth West, Riverdale",43.683178,-79.355105,0,Greek Restaurant,Café,Ice Cream Shop,Yoga Studio,Scenic Lookout,Bubble Tea Shop,Business Service,Chinese Restaurant,Fast Food Restaurant,Coffee Shop
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.667965,-79.314667,0,Brewery,Fast Food Restaurant,Café,Sandwich Place,Italian Restaurant,Coffee Shop,Bakery,Butcher,Movie Theater,Breakfast Spot
3,M4M,East Toronto,Studio District,43.660629,-79.334855,0,Coffee Shop,Café,Bakery,Italian Restaurant,Bar,Diner,American Restaurant,Pizza Place,Thai Restaurant,Fast Food Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728420,-79.387133,0,Bus Line,Gym / Fitness Center,Coffee Shop,Bookstore,Restaurant,Park,Café,Fish Market,Fish & Chips Shop,Fast Food Restaurant


# Map Including the Cluster Values for Each Zip Code/Neighborhood/Borough

In [107]:
map_clusters = folium.Map(location=[lat_tor, lng_tor], zoom_start=11)
x = np.arange(kclusters)
ys = [i + x + (i * x) ** 2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, pc, cluster in zip(df5['Latitude'], df5['Longitude'], df5['Postcode'], df5['Cluster Labels']):
    label = folium.Popup(str(pc) + ', ' + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon],
                        radius=5,
                        popup=label,
                        color=rainbow[cluster-1],
                        fill=True,
                        fill_color=rainbow[cluster-1],
                        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# most neighborhoods are the same venue-wise with the exception of two with zipcodes M4T and M4W